# 公式ドキュメントの手法(複数データの正解率)

Output: acc_document_changed.csv  

周波数バンド: **変更**(YASAと同じ周波数バンド)
|name|frequency[Hz]|
|:-|:-|
|delta|1.0-4.0|
|theta|4.0-8.0|
|alpha|8.0-12.0|
|sigma|12.0-16.0|
|beta|16.0-30.0|

In [29]:
import numpy as np
import re
import mne
import pandas as pd
from mne.datasets.sleep_physionet.age import fetch_data
from mne.time_frequency import psd_welch

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer

annotation_desc_2_event_id = {
    "Sleep stage W": 1,
    "Sleep stage 1": 2,
    "Sleep stage 2": 3,
    "Sleep stage 3": 4,
    "Sleep stage 4": 4,
    "Sleep stage R": 5,
}

# create a new event_id that unifies stages 3 and 4
event_id = {
    "Sleep stage W": 1,
    "Sleep stage 1": 2,
    "Sleep stage 2": 3,
    "Sleep stage 3/4": 4,
    "Sleep stage R": 5,
}


In [30]:
def eeg_power_band(epochs):
    """EEG relative power band feature extraction.

    This function takes an ``mne.Epochs`` object and creates EEG features based
    on relative power in specific frequency bands that are compatible with
    scikit-learn.

    Parameters
    ----------
    epochs : Epochs
        The data.

    Returns
    -------
    X : numpy array of shape [n_samples, 5]
        Transformed data.
    """
    # specific frequency bands
    FREQ_BANDS = {"delta": [1.0, 4.0],
                  "theta": [4.0, 8.0],
                  "alpha": [8.0, 12.0],
                  "sigma": [12.0, 16.0],
                  "beta": [16.0, 30]}

    psds, freqs = psd_welch(epochs, picks='eeg', fmin=0.5, fmax=30.)
    # Normalize the PSDs
    psds /= np.sum(psds, axis=-1, keepdims=True)

    X = []
    for fmin, fmax in FREQ_BANDS.values():
        psds_band = psds[:, :, (freqs >= fmin) & (freqs < fmax)].mean(axis=-1)
        X.append(psds_band.reshape(len(psds), -1))

    return np.concatenate(X, axis=1)

In [31]:
train_num = 0

[train_data] = fetch_data(subjects=[train_num], recording=[1])

raw_train = mne.io.read_raw_edf(train_data[0],
                                stim_channel='Event marker',
                                misc=['Temp rectal'])

annot_train = mne.read_annotations(train_data[1])

annot_train.crop(annot_train[1]['onset'] - 30 * 60,
                 annot_train[-2]['onset'] + 30 * 60)

raw_train.set_annotations(annot_train, emit_warning=False)

events_train, _ = mne.events_from_annotations(raw_train,
                                              event_id=annotation_desc_2_event_id,
                                              chunk_duration=30.)

tmax = 30. - 1. / raw_train.info['sfreq']  # tmax in included

epochs_train = mne.Epochs(raw=raw_train,
                          events=events_train,
                          event_id=event_id,
                          tmin=0.,
                          tmax=tmax,
                          baseline=None)


pipe = make_pipeline(FunctionTransformer(eeg_power_band, validate=False),
                     RandomForestClassifier(n_estimators=100, random_state=42))

# Train
y_train = epochs_train.events[:, 2]
pipe.fit(epochs_train, y_train)

Using default location ~/mne_data for PHYSIONET_SLEEP...
Extracting EDF parameters from /net/nfs/home/imamura22/mne_data/physionet-sleep-data/SC4001E0-PSG.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Used Annotations descriptions: ['Sleep stage 1', 'Sleep stage 2', 'Sleep stage 3', 'Sleep stage 4', 'Sleep stage R', 'Sleep stage W']
Not setting metadata
841 matching events found
No baseline correction applied
0 projection items activated
Loading data for 841 events and 3000 original time points ...
0 bad epochs dropped
Effective window size : 2.560 (s)


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.1s finished


Pipeline(steps=[('functiontransformer',
                 FunctionTransformer(func=<function eeg_power_band at 0x7fe2f1c23dc0>)),
                ('randomforestclassifier',
                 RandomForestClassifier(random_state=42))])

In [32]:
def fetch(amount):
    # [[edf1, edf2, ...], [hypno1, hypno2, ...]]
    DataList = [[], [], []]

    NAN_data = [39, 68, 69, 78, 79]
    for NAN in NAN_data:
        if amount - 1 > NAN:
            amount += 1

    for i in range(1, amount + 1):
        [data] = fetch_data(subjects=[i], recording=[1])
        edf = mne.io.read_raw_edf(data[0],
                                  stim_channel="Event marker",
                                  misc=["Temp rectal"])

        annot = mne.read_annotations(data[1])

        annot.crop(annot[1]['onset'] - 30 * 60,
                annot[-2]['onset'] + 30 * 60)

        edf.set_annotations(annot, emit_warning=False)

        events, _ = mne.events_from_annotations(
            edf, event_id=annotation_desc_2_event_id, chunk_duration=30.0
        )

        epochs = mne.Epochs(
            raw=edf,
            events=events,
            event_id=event_id,
            tmin=0.0,
            tmax=tmax,
            baseline=None,
        )

        hypno = epochs.events[:, 2] - 1
        
        DataList[0].append(epochs)
        text = re.search("SC.*-PSG.edf", str(edf))
        DataList[1].append(text)
        DataList[2].append(hypno)

    return DataList


amount = 30
Data = fetch(amount)
print(Data)

Using default location ~/mne_data for PHYSIONET_SLEEP...
Extracting EDF parameters from /net/nfs/home/imamura22/mne_data/physionet-sleep-data/SC4011E0-PSG.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Used Annotations descriptions: ['Sleep stage 1', 'Sleep stage 2', 'Sleep stage 3', 'Sleep stage 4', 'Sleep stage R', 'Sleep stage W']
Not setting metadata
1103 matching events found
No baseline correction applied
0 projection items activated
Using default location ~/mne_data for PHYSIONET_SLEEP...
Extracting EDF parameters from /net/nfs/home/imamura22/mne_data/physionet-sleep-data/SC4021E0-PSG.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Used Annotations descriptions: ['Sleep stage 1', 'Sleep stage 2', 'Sleep stage 3', 'Sleep stage 4', 'Sleep stage R', 'Sleep stage W']
Not setting metadata
1025 matching events found
No baseline correction applied
0 projection items activated
Using default location ~/m

In [33]:
accuracy = []

for i in range(amount):
    # Test
    hypno_pred = pipe.predict(Data[0][i]) - 1

    hypno = Data[2][i]

    try:
        acc = accuracy_score(hypno, hypno_pred)  # 正解率
        print(str(Data[0][i]))
        text = Data[1][i]
        tmp = [acc, text.group()]
        accuracy.append(tmp)
    except ValueError:
        pass

csv_pre = pd.DataFrame(accuracy)
csv_pre.to_csv("./acc_document_changed.csv")


Loading data for 1103 events and 3000 original time points ...
0 bad epochs dropped
Effective window size : 2.560 (s)


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


<Epochs |  1103 events (all good), 0 - 29.99 sec, baseline off, ~12 kB, data not loaded,
 'Sleep stage W': 157
 'Sleep stage 1': 109
 'Sleep stage 2': 562
 'Sleep stage 3/4': 105
 'Sleep stage R': 170>
Loading data for 1025 events and 3000 original time points ...


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.4s finished


0 bad epochs dropped
Effective window size : 2.560 (s)


...

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.2s finished


0 bad epochs dropped
Effective window size : 2.560 (s)


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


<Epochs |  929 events (all good), 0 - 29.99 sec, baseline off, ~12 kB, data not loaded,
 'Sleep stage W': 211
 'Sleep stage 1': 109
 'Sleep stage 2': 457
 'Sleep stage 3/4': 30
 'Sleep stage R': 122>


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.0s finished
